In [13]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
import lxml
import wget
import zipfile
from datetime import datetime

In [2]:
# Set up splinter because "view source" doesn't match "inspect"

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_win32.zip
Driver has been 

In [3]:
# URL of page to be scraped
url = "https://s3.amazonaws.com/tripdata/index.html"

# Visit the browser
browser.visit(url)

In [4]:
# DO NOT merge this cell with the cell above (url=...), the web page needs time to load all the links in
# Set up BeautifulSoup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# Find the tag containing the desired information
results = soup.find_all('a', class_='')
results[0]

<a href="https://s3.amazonaws.com/tripdata/201306-citibike-tripdata.zip">201306-citibike-tripdata.zip</a>

In [6]:
# Create list of years, and names for the subsequent lists
years_list = []
names_list = []

for i in range(0, (len(results) - 1 )): # the "-1" will filter out index.html
    
    # extract the date
    date = results[i]["href"].split('/')[-1].split('-')[0]
    
    # filter out Jersey City (JC) data
    if (date != "JC"):
        
        # Extract the year
        year = int(date[0:4])
                
        if (year not in years_list):
            
            # Add the found year to the years_list
            years_list.append(year)
            
            # Create name, append to list
            name = f"data_{year}"
            names_list.append(name)

In [7]:
for name in names_list:
    globals()[f"{name}"] = []

In [8]:
# Loop through every link in the list
for i in range(0, (len(results) - 1 )): # the "-1" will filter out index.html
    
    # Find the date
    date = results[i]["href"].split('/')[-1].split('-')[0]
    
    # filter out Jersey City (JC) data
    if (date != "JC"):
        
        # Find the year
        year = int(date[0:4])

        # Loop through every list
        for j in range (0, len(names_list)):

            if (year == years_list[j]):
                
                # Append any applicable link to the list
                globals()[f"{names_list[j]}"].append(results[i]["href"])

In [20]:
year_selection = input(f"Choose a year between {years_list[0]} and {years_list[-1]}:")

Choose a year between 2013 and 2022: 2022


In [21]:
selected_data = globals()[f"data_{year_selection}"]

# Limit can be reduced for testing purposes, use len(selected_data) to run the remaining code for all files returned
limit = len(selected_data)

In [22]:
# Create lists of zip names and csv names to use when calling the file
zip_names = []
csv_names = []

# for entry in results:
for i in range(0, limit):
    
    # Get the link
    link = selected_data[i]

    # Use the link to get the zip and csv names, in case the .text is mismatched
    zip_name = link.split('/')[-1]
    zip_names.append(zip_name)

    csv_name = zip_name.split('.')[0] + ".csv"
    csv_names.append(csv_name)
   
    # Download the zip
    wget.download(link, out = "zipfiles") 


100% [....................................................] 66434482 / 66434482

In [23]:
#Create CSVs

#Set up the station information data frame and tracker list
station_df = pd.DataFrame(columns = ['station_name', 'station_id', 'latitude', 'longitude'])
station_names = []

for i in range(0, limit):
    
    # Drop nulls
    df = df.dropna()
    
    # find and read the zip file
    find_zip = "zipfiles/" + zip_names[i]
    zf = zipfile.ZipFile(find_zip)
    df = pd.read_csv(zf.open(csv_names[i]))

    # Add to the station_data df
    for index, row in df.iterrows():
        # Capturing station information from the start station
        if row["start_station_name"] not in station_names:
            start_station_name = row["start_station_name"] 
            start_station_id = row["start_station_id"] 
            start_lat = row["start_lat"]
            start_lng = row["start_lng"]
            
            # Append a new row
            station_df = station_df.append({'station_name': start_station_name, 'station_id': start_station_id, 'latitude': start_lat, 'longitude': start_lng}, ignore_index = True)

            # Add to the tracker list
            station_names.append(start_station_name)

    # Create new, smaller dataframe with extraneous station data removed
    reduced_df = df[["ride_id", "rideable_type", "started_at", "ended_at", "start_station_id", "end_station_id", "member_casual"]].copy()
    
    # Create a trip duration column
    reduced_df["ended_at"] = pd.to_datetime(reduced_df["ended_at"], format='%Y-%m-%d %H:%M:%S')
    reduced_df["started_at"] = pd.to_datetime(reduced_df["started_at"], format='%Y-%m-%d %H:%M:%S')
    reduced_df["trip_duration_s"] = (reduced_df["ended_at"] - reduced_df["started_at"]).dt.total_seconds()
     
    #create the csv
    reduced_df.to_csv("data/" + csv_names[i], index = False)

#Save the station dataframe as a CSV
station_df.to_csv("data/station_data.csv")

C:\Users\emera\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\emera\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\emera\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [19]:
reduced_df

,ride_id,rideable_type,started_at,ended_at,start_station_id,end_station_id,member_casual,trip_duration_s
0,72D1C9F1A2FEB359,classic_bike,2021-12-04 17:53:23,2021-12-04 18:05:37,4695.04,4713.01,casual,734.0
1,8BE7469E1A90508C,electric_bike,2021-12-04 17:35:04,2021-12-04 17:52:55,6215.04,6416.06,casual,1071.0
2,AF9EDCC2A347E607,electric_bike,2021-12-13 23:59:41,2021-12-14 00:08:31,4695.04,5002.09,casual,530.0
3,495CF3C8A1B7FED8,classic_bike,2021-12-22 12:22:38,2021-12-22 12:25:46,7627.1,7617.07,member,188.0
4,1C138E3D55FCC7C1,classic_bike,2021-12-16 10:15:22,2021-12-16 10:32:02,6382.05,5569.06,member,1000.0
...,...,...,...,...,...,...,...,...
1748282,714D3CCFB28DC79C,classic_bike,2021-12-06 15:33:18,2021-12-06 15:42:18,6239.08,5947.04,member,540.0
1748283,1A2F3968E4A735FB,docked_bike,2021-12-11 14:24:00,2021-12-12 13:58:42,5797.01,5593.02,casual,84882.0
1748284,1BD68D08120346F9,classic_bike,2021-12-05 14:15:22,2021-12-05 14:23:28,4565.04,4413.08,member,486.0
1748285,F7D2A9EAE112CF48,classic_bike,2021-12-31 11:22:45,2021-12-31 11:57:19,4683.02,3955.05,member,2074.0


In [40]:

station_df = pd.DataFrame(columns = ['station_name', 'station_id', 'latitude', 'longitude'])

station_names = []

for index, row in df.iterrows():

    
    # Capturing station information from the start station
    if row["start_station_name"] not in station_names:
        start_station_name = row["start_station_name"] 
        start_station_id = row["start_station_id"] 
        start_lat = row["start_lat"]
        start_lng = row["start_lng"]
    
        station_df = station_df.append({'station_name': start_station_name, 'station_id': start_station_id, 'latitude': start_lat, 'longitude': start_lng}, ignore_index = True)
    
        station_names.append(start_station_name)

station_df

,station_name,station_id,latitude,longitude
0,Washington St & Barrow St,5847.08,40.731911,-74.008769
1,1 Ave & E 110 St,7522.02,40.792327,-73.938300
2,Crescent St & Broadway,6827.11,40.763359,-73.928647
3,Stagg St & Union Ave,5117.05,40.708771,-73.950953
4,Frost St & Meeker Ave,5371.07,40.717662,-73.948800
...,...,...,...,...
1539,Cauldwell Ave & E 158 St,7913.15,40.819890,-73.908351
1540,Townsend Ave & E 175 St,8323.05,40.847410,-73.911359
1541,63 St & 5 Ave,2872.02,40.637660,-74.017820
1542,NaN,NaN,40.670000,-73.970000


In [42]:
station_df.to_csv("data/station_data.csv")

In [17]:
csv_names[0]

'202201-citibike-tripdata.csv'

### Next up: Turn all station info into a separate smaller csv/table, remove NULL values, ...remove outliers?

In [ ]:
browser.quit()

### OLD Code with non-dynamic list names

In [6]:
data_2013to2016 = []
data_2017 = []
data_2018 = []
data_2019 = []
data_2020 = []
data_2021 = []
data_2022 = []


for i in range(0, (len(results) - 1 )): # the "-1" will filter out index.html
    date = results[i]["href"].split('/')[-1].split('-')[0]
    
    # filter out Jersey City (JC) data
    if (date != "JC"):
        year = int(date[0:4])
        if (year <= 2016):
            data_2013to2016.append(results[i]["href"])
        elif (year == 2017):
            data_2017.append(results[i]["href"])
        elif (year == 2018):
            data_2018.append(results[i]["href"])
        elif (year == 2019):
            data_2019.append(results[i]["href"])
        elif (year == 2020):
            data_2020.append(results[i]["href"])
        elif (year == 2021):
            data_2021.append(results[i]["href"])
        elif (year == 2022):
            data_2022.append(results[i]["href"])
           
            
data_2022

['https://s3.amazonaws.com/tripdata/202201-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/202202-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/202203-citibike-tripdata.csv.zip']

In [7]:
# Download zip files for a given year or year set
data_list = data_2022

limit = len(data_list)

In [8]:
# Create lists of zip names and csv names to use when calling the file
zip_names = []
csv_names = []

# for entry in results:
for i in range(0, limit):
    
    # Get the link
    link = data_list[i]

    # Use the link to get the zip and csv names, in case the .text is mismatched
    zip_name = link.split('/')[-1]
    zip_names.append(zip_name)

    csv_name = zip_name.split('.')[0] + ".csv"
    csv_names.append(csv_name)
   
    # Download the zip
    wget.download(link, out = "zipfiles") 
    
#     # Open and save as CSV
#     df = pd.read_csv(zf.open(csv_names[i]))
#     df.name = csv_names[i]
#     df.to_csv("CSVs")
    
    

100% [....................................................] 66434482 / 66434482

In [9]:
#Save as CSVs

for i in range(0, limit):
    
    # find and read the zip file
    find_zip = "zipfiles/" + zip_names[i]
    zf = zipfile.ZipFile(find_zip)
    df = pd.read_csv(zf.open(csv_names[i]))
    
    # name and create the csv
    df.name = csv_names[i]
    df.to_csv("data/" + csv_names[i])
    
    

C:\Users\emera\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
df = pd.read_csv("data/202112-citibike-tripdata.csv")
df["ended_at"] = pd.to_datetime(df["ended_at"], format='%Y-%m-%d %H:%M:%S')
df["started_at"] = pd.to_datetime(df["started_at"], format='%Y-%m-%d %H:%M:%S')

In [17]:
df["trip_duration_s"] = (df["ended_at"] - df["started_at"]).dt.total_seconds()
df["trip_duration_s"]

0            734.0
1           1071.0
2            530.0
3            188.0
4           1000.0
            ...   
1748282      540.0
1748283    84882.0
1748284      486.0
1748285     2074.0
1748286      730.0
Name: trip_duration_s, Length: 1748287, dtype: float64

In [17]:
df = df.dropna()
df.count()

ride_id             1739723
rideable_type       1739723
started_at          1739723
ended_at            1739723
start_station_id    1739723
end_station_id      1739723
member_casual       1739723
dtype: int64

In [12]:
df = pd.read_csv("data/station_data.csv")
df.count()

Unnamed: 0      1570
station_name    1569
station_id      1569
latitude        1570
longitude       1570
dtype: int64

In [13]:
df = df.dropna()
df.count()

Unnamed: 0      1569
station_name    1569
station_id      1569
latitude        1569
longitude       1569
dtype: int64

In [9]:
df = df[df["station_name"].str.strip().astype(bool)]

AttributeError: 'DataFrame' object has no attribute 'str'